# **About**

|Section|Details|
|---|---|
|Script|rs3-request-data|
|Description|rs3-request-data can manually be used to retrieve historical price data for items in Runescape 3. This data will be transformed to predict future item prices.|
|Author|Andrew Yang|

# **Setup**

In [31]:
import requests # use to retrieve data from API
import math
import pandas as pd
import datetime
#from datetime import datetime, timedelta, UTC
#import os

In [2]:
# Global var for debugging.
show_debug = True

# Global var for integrating social media information into the model.
integrate_social = False

# Global vars for Weird Gloop API.
game_base = "rs" # API can have two options: rs (Runescape 3) or osrs (Old School Runescape)
data_filter = "all" # API has three options: all (all price data), last90d (last 90 days), and sample

# Determine which item category we are interested in.
# See the following link for all item categories (https://runescape.wiki/w/Application_programming_interface#category).
item_category = 13 

# **Data Gathering**

##  Retrieve items by category

### Get item categories

In [3]:
# See the following link for a description of all item categories (https://runescape.wiki/w/Application_programming_interface#category).
# i.e. 0 = Miscellaneous; 7 = Costumes; 13 = Herblore materials 
category_ids = range(0, 44, 1)
print(f"All category ids: {list(category_ids)}")

All category ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


### Retrieve items in category

In [4]:
# Grabs a list of dictionaries that show how many items are under each "alpha" (character)
request_category_base = "https://services.runescape.com/m=itemdb_rs/api/catalogue/category.json?"
r_category = requests.get(request_category_base, params = {"category": item_category})
print(r_category.status_code)

200


In [5]:
category_alpha_dict = r_category.json()["alpha"]

print(f"Item alpha dict: ")
display(category_alpha_dict)

Item alpha dict: 


[{'letter': '#', 'items': 0},
 {'letter': 'a', 'items': 6},
 {'letter': 'b', 'items': 7},
 {'letter': 'c', 'items': 32},
 {'letter': 'd', 'items': 5},
 {'letter': 'e', 'items': 1},
 {'letter': 'f', 'items': 2},
 {'letter': 'g', 'items': 29},
 {'letter': 'h', 'items': 2},
 {'letter': 'i', 'items': 2},
 {'letter': 'j', 'items': 3},
 {'letter': 'k', 'items': 2},
 {'letter': 'l', 'items': 6},
 {'letter': 'm', 'items': 13},
 {'letter': 'n', 'items': 1},
 {'letter': 'o', 'items': 0},
 {'letter': 'p', 'items': 6},
 {'letter': 'q', 'items': 0},
 {'letter': 'r', 'items': 5},
 {'letter': 's', 'items': 12},
 {'letter': 't', 'items': 5},
 {'letter': 'u', 'items': 2},
 {'letter': 'v', 'items': 3},
 {'letter': 'w', 'items': 6},
 {'letter': 'x', 'items': 0},
 {'letter': 'y', 'items': 2},
 {'letter': 'z', 'items': 1}]

In [6]:
# Define API endpoint for getting item info.
request_items_base = "https://services.runescape.com/m=itemdb_rs/api/catalogue/items.json?"

def get_item_details(req_category, req_alpha, req_page):
    r_items = requests.get(request_items_base, params = {"category": req_category, "alpha": req_alpha, "page": req_page})
    if show_debug:
        print(f"Status of item category|alpha|page ({req_category}|{req_alpha}|{req_page}): {r_items.status_code}")
    return r_items

In [7]:
item_dict_list = {}

# Add entries to a dictionary linking an item id to item name.
def get_item_id(item_request):
    for item in item_request.json()["items"]:
        item_dict_list[item["id"]] = item["name"]

In [8]:
for a in category_alpha_dict:
    if a["items"] <= 0: 
        continue
    
    req_requests =  math.ceil(a["items"] / 12) # Determines the # of times to request the API for all items.
    for page in range(1, req_requests + 1, 1):
        get_item_id(get_item_details(item_category, a["letter"], page))

Status of item category|alpha|page (13|a|1): 200
Status of item category|alpha|page (13|b|1): 200
Status of item category|alpha|page (13|c|1): 200
Status of item category|alpha|page (13|c|2): 200
Status of item category|alpha|page (13|c|3): 200
Status of item category|alpha|page (13|d|1): 200
Status of item category|alpha|page (13|e|1): 200
Status of item category|alpha|page (13|f|1): 200
Status of item category|alpha|page (13|g|1): 200
Status of item category|alpha|page (13|g|2): 200
Status of item category|alpha|page (13|g|3): 200
Status of item category|alpha|page (13|h|1): 200
Status of item category|alpha|page (13|i|1): 200
Status of item category|alpha|page (13|j|1): 200
Status of item category|alpha|page (13|k|1): 200
Status of item category|alpha|page (13|l|1): 200
Status of item category|alpha|page (13|m|1): 200
Status of item category|alpha|page (13|m|2): 200
Status of item category|alpha|page (13|n|1): 200
Status of item category|alpha|page (13|p|1): 200
Status of item categ

In [9]:
# print(f"Item id dictionary ({len(item_dict_list)} entries):")
# display(item_dict_list)

##  Retrieving item GE prices from API

In [10]:
# The RS3 Wiki uses Weird Gloop for their API, and includes support for retrieving their stored Grand Exchange data.
# We can avoid request limitations with Jagex's own Grand Exchange API, and get all historical data.
request_prices_base = f"https://api.weirdgloop.org/exchange/history/{game_base}/{data_filter}"
print(f"Historical GE prices request endpoint: {request_prices_base}")

Historical GE prices request endpoint: https://api.weirdgloop.org/exchange/history/rs/all


In [11]:
def get_historical_prices(item_filter):
    r_prices = requests.get(request_prices_base, params = {"id": item_filter})
    if show_debug:
        print(f"Status of item {item_filter} ({item_dict_list[item_filter]}):{r_prices.status_code}")
    return r_prices.json()[f"{item_filter}"]

In [12]:
ge_prices = []
for item_id_entry in item_dict_list.keys():
    ge_prices.extend(get_historical_prices(item_id_entry))

Status of item 52937 (Abyssal flesh):200
Status of item 39067 (Adrenaline crystal):200
Status of item 48241 (Arbuck potion (unf)):200
Status of item 592 (Ashes):200
Status of item 103 (Avantoe potion (unf)):200
Status of item 48578 (Avocado):200
Status of item 48925 (Beak snot):200
Status of item 37973 (Bloodweed potion (unf)):200
Status of item 243 (Blue dragon scale):200
Status of item 48961 (Bomb vial):200
Status of item 48926 (Bottled dinosaur roar):200
Status of item 4456 (Bowl of hot water):200
Status of item 43993 (Bull horns):200
Status of item 6016 (Cactus spine):200
Status of item 107 (Cadantine potion (unf)):200
Status of item 48586 (Carambola):200
Status of item 11326 (Caviar):200
Status of item 43973 (Chinchompa residue):200
Status of item 48584 (Ciku):200
Status of item 48211 (Clean arbuck):200
Status of item 261 (Clean avantoe):200
Status of item 37953 (Clean bloodweed):200
Status of item 265 (Clean cadantine):200
Status of item 267 (Clean dwarf weed):200
Status of item 

In [13]:
#display(ge_prices)

## Consolidate data into dataframe

In [14]:
# Convert our price list of dictionaries into a Pandas Dataframe.
ge_df = pd.DataFrame(ge_prices)

In [15]:
#display(ge_df)

# Data supplementation

In [16]:
# Checking how many null values are there - will most likely be within the volume.
ge_df.isna().sum() / len(ge_df)

id           0.000000
price        0.000000
volume       0.839791
timestamp    0.000000
dtype: float64

## Improve human readability

In [17]:
# Convert unix timestamp to date.
def unix_to_date(ts):
    return datetime.datetime.fromtimestamp(ts/1000, datetime.UTC).strftime('%Y-%m-%d')

In [18]:
# Determine if a unix timestamp represents a weekend or weekday.
def unix_is_weekday(ts):
    return 1 if datetime.datetime.fromtimestamp(ts/1000, datetime.UTC).weekday() < 5 else 0

In [19]:
# Adds name of item based on item id.
def id_to_name(item_id):
    return item_dict_list[int(item_id)]

In [20]:
# Make human-readable date, item name, and whether or not said date is a weekday.
ge_df["date"] = ge_df['timestamp'].map(unix_to_date)
ge_df["weekday"] = ge_df['timestamp'].map(unix_is_weekday)
ge_df["name"] = ge_df['id'].map(id_to_name)

## Add differenced time series

In [21]:
# Get differenced data by 1 day, 1 week, 2 weeks, and ~ 1 month (comparing price for each item by their id). Great for time series.
ge_df["diff_1_day"] = ge_df.groupby("id")["price"].diff(1)
ge_df["diff_7_day"] = ge_df.groupby("id")["price"].diff(7)
ge_df["diff_14_day"] = ge_df.groupby("id")["price"].diff(14)
ge_df["diff_30_day"] = ge_df.groupby("id")["price"].diff(30)

## Add moving average time series

In [22]:
# Get 1 week, 2 week, and ~1 month moving average (comparing price for each item by their id).
ge_df["ma_7_day"] = ge_df.groupby("name")["price"].rolling(7, 1).mean().reset_index(drop=True)
ge_df["ma_14_day"] = ge_df.groupby("name")["price"].rolling(14, 1).mean().reset_index(drop=True)
ge_df["ma_30_day"] = ge_df.groupby("name")["price"].rolling(30, 1).mean().reset_index(drop=True)

## Clean price time series

In [23]:
# Get the difference between 1 week, 2 week, and ~1 month moving average for each day.
ge_df["diff_ma_7_day"] = ge_df.groupby("id")["ma_7_day"].diff(1)
ge_df["diff_ma_14_day"] = ge_df.groupby("id")["ma_14_day"].diff(1)
ge_df["diff_ma_30_day"] = ge_df.groupby("id")["ma_30_day"].diff(1)

In [24]:
# Create the following conditions to filter ge_df:
# cond1 - If the difference in moving average at a dat for 1 week, 2 weeks, and 1 month are = 0, the date's price is likely the initial Jagex-set price.
# cond2 - Similarly, remove rows with null values in difference in moving average.
# cond3 - Finally, as a precaution, remove null rows for price differences at 1 day, 1 week, 2 weeks, and 1 month.

cond1 = (ge_df["diff_ma_7_day"] != 0) & (ge_df["diff_ma_14_day"] != 0) & (ge_df["diff_ma_30_day"] != 0)
cond2 = (ge_df["diff_ma_7_day"].notnull()) & (ge_df["diff_ma_14_day"].notnull()) & (ge_df["diff_ma_30_day"].notnull())
cond3 = (ge_df["diff_1_day"].notnull()) & (ge_df["diff_7_day"].notnull()) & (ge_df["diff_14_day"].notnull())& (ge_df["diff_30_day"].notnull())

# Filter original ge prices dataframe based on the above conditions to get clean values.
ge_enriched_df = ge_df[cond1 & cond2 & cond3].copy().reset_index()
display(ge_enriched_df.sample(10, random_state = 123))

,index,id,price,volume,timestamp,date,weekday,name,diff_1_day,diff_7_day,diff_14_day,diff_30_day,ma_7_day,ma_14_day,ma_30_day,diff_ma_7_day,diff_ma_14_day,diff_ma_30_day
35644,44574,6016,6519,NaN,1332720000000,2012-03-26,1,Cactus spine,-54.0,69.0,265.0,-384.0,6618.571429,6496.714286,6649.433333,9.857143,18.928571,-12.800000
111962,127221,251,108,NaN,1423180800000,2015-02-06,1,Clean marrentill,-2.0,10.0,34.0,17.0,106.000000,97.071429,93.700000,1.428571,2.428571,0.566667
254158,294319,209,3076,NaN,1277424000000,2010-06-25,1,Grimy irit,0.0,-89.0,-208.0,46.0,3093.714286,3175.714286,3122.200000,-12.714286,-14.857143,1.533333
458725,615521,43997,621,NaN,1595116800000,2020-07-19,0,Spider venom,-24.0,-174.0,-168.0,110.0,700.142857,722.285714,710.966667,-24.857143,-12.000000,3.666667
453500,609648,231,353,NaN,1663718400000,2022-09-21,1,Snape grass,-1.0,-1.0,43.0,68.0,355.857143,344.500000,322.466667,-0.142857,3.071429,2.266667
399923,537646,43995,1035,14456.0,1682162622000,2023-04-22,0,Mycelial webbing,0.0,59.0,142.0,176.0,1027.857143,988.142857,935.066667,8.428571,10.142857,5.866667
485333,645220,95,176,NaN,1529107200000,2018-06-16,0,Tarromin potion (unf),0.0,6.0,8.0,3.0,172.571429,171.000000,169.400000,0.857143,0.571429,0.100000
442626,596028,7650,819,NaN,1514851200000,2018-01-02,1,Silver dust,0.0,-23.0,-23.0,7.0,819.000000,830.500000,835.633333,-3.285714,-1.642857,0.233333
547130,721375,245,12234,NaN,1610496000000,2021-01-13,1,Wine of Zamorak,-243.0,837.0,257.0,1318.0,12300.714286,11927.357143,11902.766667,119.571429,18.357143,43.933333
296324,337391,203,35,NaN,1400803200000,2014-05-23,1,Grimy tarromin,1.0,-3.0,-8.0,-6.0,34.571429,37.214286,39.533333,-0.428571,-0.571429,-0.200000


## Get social media sourced information

In [25]:
# The RS3 Wiki uses Weird Gloop for their API; this endpoint gets all social media information.
request_socials_base = f"https://api.weirdgloop.org/runescape/social"
halt = False # Used in while loop - API response includes if there are additional pages left.
max_iter = 100 # Failsafe.

In [26]:
page = 1
social_dict_list = []

while integrate_social != False and halt != True and page <= max_iter: # As long as we haven't halted the process and page less than the max allowed
    r_social = requests.get(request_socials_base, params = {"page": page}) # Request social media info from API.
    
    if show_debug:
        print(f"Status of page {page}: {r_social.status_code}")
    page += 1

    if r_social.json()["pagination"]["has_more"] != True: # If the response tells use there's no more pages, halt the loop.
        halt = True

    social_dict_list.extend(r_social.json()["data"]) # Add dictionaries to our list.

In [27]:
# Get social media list of dicts into a dataframe.
social_df = pd.DataFrame(social_dict_list)

In [28]:
# Convert odd datetime format to string, and get date.
def datetime_to_string(dt):
    return str(dt)[:10]

In [29]:
# Create string version of date to link social media info to specific dates.
social_df["date"] = social_df["dateAdded"].map(datetime_to_string)

KeyError: 'dateAdded'

In [30]:
# Enrich social media history dataframe based on title of media item:
#   Launch usually indicates a new release.
#   Bosses are big drop sources of items, and may affect our items of interest.
#   Quests show info about an upcoming Runescape quest. These quests may unlock new things which require our items of interest.
#   Event usually indicates a new upcoming events.
#   Double XP tells players when the next Double XP is coming up, and is a known market mover.
#   Update is more general, but can include information on changes for any of the above info... or something irrelevant.
social_df["launch_update"] = social_df["title"].apply(lambda x: "Launch" in x if x is not None else False)
social_df["boss_update"] = social_df["title"].apply(lambda x: "Boss" in x if x is not None else False)
social_df["quest_update"] = social_df["title"].apply(lambda x: "Quest" in x if x is not None else False)
social_df["event_update"] = social_df["title"].apply(lambda x: "Event" in x if x is not None else False)
social_df["dxp_update"] = social_df["title"].apply(lambda x: "Double XP" in x if x is not None else False)
social_df["general_update"] = social_df["title"].apply(lambda x: "Update" in x if x is not None else False)

In [31]:
ge_dates = sorted(ge_enriched_df.date.unique())
print(f"Earliest GE date: {ge_dates[0]}; Most recent GE date: {ge_dates[-1]}.")

earliest_update_date = social_df.date.min()
recent_update_date = social_df.date.max()
print(f"Earliest update: {earliest_update_date}; Most recent update: {recent_update_date}.")

Earliest GE date: 2008-06-20; Most recent GE date: 2024-09-21.
Earliest update: 2020-02-14; Most recent update: 2024-09-19.


In [32]:
# Get range of 
max_update_date = max(recent_update_date, ge_dates[-1])
min_update_date = max(earliest_update_date, ge_dates[0])
print(f"Earliest simulated date: {min_update_date}; Most recent simulated date: {max_update_date}.")

sim_min_date = datetime.datetime.strptime(min_update_date, '%Y-%m-%d')
sim_max_date = datetime.datetime.strptime(max_update_date, '%Y-%m-%d')

Earliest simulated date: 2020-02-14; Most recent simulated date: 2024-09-21.


In [33]:
# Sets iteration range.
range_days = pd.date_range(sim_min_date, sim_max_date).to_list()
print(f"Iterate over {len(range_days)} days.")

update_concat_list = []
for d in range_days:
    update_concat_list.append([d.strftime('%Y-%m-%d'), False, False, False, False, False, False])

print(update_concat_list[0])

Iterate over 1682 days.
['2020-02-14', False, False, False, False, False, False]


In [34]:
social_temp_df = social_df[['date', 'launch_update', 'quest_update', 'event_update', 'dxp_update', 
                            'general_update', 'boss_update']]
social_append_df = pd.DataFrame(update_concat_list, columns = social_temp_df.columns)
social_enriched_df = pd.concat([social_temp_df, social_append_df], axis = 0)
display(social_enriched_df.sample(10, random_state = 123))

,date,launch_update,quest_update,event_update,dxp_update,general_update,boss_update
263,2022-08-25,False,False,False,False,False,False
239,2022-11-02,False,False,False,False,False,False
379,2021-10-04,False,False,False,False,False,False
716,2020-04-02,False,False,False,False,False,False
85,2024-01-22,False,False,False,False,False,False
483,2021-06-11,False,False,False,False,False,False
1438,2024-01-22,False,False,False,False,False,False
810,2022-05-04,False,False,False,False,False,False
233,2020-10-04,False,False,False,False,False,False
24,2020-03-09,False,False,False,False,False,False


In [35]:
# Aggregate by date to determine if an update occured on or around a specific date.
social_agg_df = social_enriched_df.groupby(["date"], as_index=False).any()
#display(social_agg_df)

In [36]:
# Calculate moving averages for 7, 14, and 30 days and put into dataframes.
# FUTURE UPDATE - experiment with different window weight methods (like exponential or gaussian), to represent decaying influence of update.
social_agg_df_7_ma = social_agg_df[social_agg_df.columns[social_agg_df.columns!='date']].rolling(7, 1).mean()
social_agg_df_14_ma = social_agg_df[social_agg_df.columns[social_agg_df.columns!='date']].rolling(14, 1).mean()
social_agg_df_30_ma = social_agg_df[social_agg_df.columns[social_agg_df.columns!='date']].rolling(30, 1).mean()

In [37]:
# Rename columns for clarity.
social_agg_df_7_ma.columns = [f"{c}_7_ma" for c in social_agg_df_7_ma.columns.to_list()]
social_agg_df_14_ma.columns = [f"{c}_14_ma" for c in social_agg_df_14_ma.columns.to_list()]
social_agg_df_30_ma.columns = [f"{c}_30_ma" for c in social_agg_df_30_ma.columns.to_list()]

In [38]:
# Join together dataframes of original data + moving averages.
social_agg_final_df = pd.concat([social_agg_df, social_agg_df_7_ma, social_agg_df_14_ma, social_agg_df_30_ma], axis=1)
display(social_agg_final_df)

,date,launch_update,quest_update,event_update,dxp_update,general_update,boss_update,launch_update_7_ma,quest_update_7_ma,event_update_7_ma,...,event_update_14_ma,dxp_update_14_ma,general_update_14_ma,boss_update_14_ma,launch_update_30_ma,quest_update_30_ma,event_update_30_ma,dxp_update_30_ma,general_update_30_ma,boss_update_30_ma
0,2020-02-14,False,False,False,False,False,False,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,2020-02-15,False,False,False,False,False,False,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,2020-02-16,False,False,False,False,False,False,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,2020-02-17,False,False,False,False,False,False,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,2020-02-18,False,False,False,False,False,False,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,2024-09-17,False,False,False,False,False,False,0.142857,0.285714,0.0,...,0.0,0.0,0.0,0.000000,0.066667,0.1,0.0,0.0,0.0,0.000000
1678,2024-09-18,False,False,False,False,False,False,0.142857,0.285714,0.0,...,0.0,0.0,0.0,0.000000,0.033333,0.1,0.0,0.0,0.0,0.000000
1679,2024-09-19,False,False,False,False,False,True,0.142857,0.142857,0.0,...,0.0,0.0,0.0,0.071429,0.033333,0.1,0.0,0.0,0.0,0.033333
1680,2024-09-20,False,False,False,False,False,False,0.142857,0.142857,0.0,...,0.0,0.0,0.0,0.071429,0.033333,0.1,0.0,0.0,0.0,0.033333


## Join in social media info

In [39]:
# Determine final ge price data, based on whether or not to integrate social media.
ge_final_df = ge_enriched_df
if integrate_social:
    ge_final_df = ge_enriched_df.merge(social_agg_final_df, left_on='date', right_on='date', how = "left")

# View results

In [40]:
display(ge_final_df)

,index,id,price,volume,timestamp,date,weekday,name,diff_1_day,diff_7_day,diff_14_day,diff_30_day,ma_7_day,ma_14_day,ma_30_day,diff_ma_7_day,diff_ma_14_day,diff_ma_30_day
0,180,52937,1653,NaN,1646092800000,2022-03-01,1,Abyssal flesh,153.0,153.0,153.0,153.0,1521.857143,1510.928571,1505.100000,21.857143,10.928571,5.100000
1,181,52937,1736,NaN,1646179200000,2022-03-02,1,Abyssal flesh,83.0,236.0,236.0,236.0,1555.571429,1527.785714,1512.966667,33.714286,16.857143,7.866667
2,182,52937,1823,NaN,1646265600000,2022-03-03,1,Abyssal flesh,87.0,323.0,323.0,323.0,1601.714286,1550.857143,1523.733333,46.142857,23.071429,10.766667
3,183,52937,1915,NaN,1646352000000,2022-03-04,1,Abyssal flesh,92.0,415.0,415.0,415.0,1661.000000,1580.500000,1537.566667,59.285714,29.642857,13.833333
4,184,52937,2011,NaN,1646438400000,2022-03-05,0,Abyssal flesh,96.0,511.0,511.0,511.0,1734.000000,1617.000000,1554.600000,73.000000,36.500000,17.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556524,732036,43983,367,11233.0,1726567520000,2024-09-17,1,Zygomite fruit,9.0,-9.0,-49.0,-150.0,361.428571,376.428571,421.166667,-1.285714,-3.500000,-5.000000
556525,732037,43983,367,5972.0,1726663524000,2024-09-18,1,Zygomite fruit,0.0,4.0,-49.0,-152.0,362.000000,372.928571,416.100000,0.571429,-3.500000,-5.066667
556526,732038,43983,377,14532.0,1726754288000,2024-09-19,1,Zygomite fruit,10.0,14.0,-22.0,-142.0,364.000000,371.357143,411.366667,2.000000,-1.571429,-4.733333
556527,732039,43983,377,5382.0,1726808841000,2024-09-20,1,Zygomite fruit,0.0,14.0,-22.0,-120.0,366.000000,369.785714,407.366667,2.000000,-1.571429,-4.000000


# Export data

In [41]:
# Create file based on date.
today = datetime.date.today().strftime('%Y-%m-%d')
print(f"Current date: {today}")

filename = f"ge-prices-{today}.csv"
print(f"Final filename: {filename}")

Current date: 2024-09-22
Final filename: ge-prices-2024-09-22.csv


In [43]:
# Export file.
ge_final_df.to_csv(filename, index=False)  